# Simple probabilistic epidemic model
## based on the deterministic SIR model, following Kermack & McKendrick (1927), combined with Monte-Carlo simulations

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interactive, HBox, VBox
import ipywidgets as widgets


In [3]:
def SIR(x, t, Ro, tau, d, t_start_qua, t_stop_qua, Ro_qua):
    if (t >= t_start_qua / tau) & (t <= t_stop_qua / tau):
        Ro_t = Ro_qua
    else:
        Ro_t = Ro
    S, I, R, P = x
    xdot = np.array([-Ro_t * S * I, Ro_t * S * I - I, (1. - d) * I, -d * I])
    return xdot


def solve(N, L, x0, Ro, tau, d, t_start_qua, t_stop_qua, Ro_qua):
    t = np.linspace(0, L, N) / tau
    x = odeint(SIR, x0, t, args=(Ro, tau, d, t_start_qua, t_stop_qua, Ro_qua))
    # output
    x = x * 100.  # convert to %
    return x.T


In [4]:
%matplotlib widget


params = dict(
    N=120,  # N.o. timesteps
    L=400,  # length run (days)
    t_start_qua=0.,  # time quarantine starts (days)
    t_stop_qua=0.1,  # time quarantine ends (days)
    # Initial conditions
    x0=np.array([0.999, 0.001, 0., 1.]),
    Ro=2.5,
    tau=14.,  # duration of infection (days)
    d=0.,  # death rate (% infected who die)
    Ro_qua=2.5,  # Ro during quarantine
)
uncertain_param_names = ['Ro', 'tau', 'd', 'Ro_qua']
assert (all(key in params for key in uncertain_param_names))

percs = [95, 90, 75, 50, 25, 10, 5]
linestyle_percs = [':','--','-.','-','-.','--',':']
stdev = 0.1  # stdev for monte carlo sampling (arbitrary!)
N_mc = 200

plot_single_sims = False # whether to plot all simulations. dont do this when N_mc is large
def solve_mc(params, stdev):
    res = []
    for _ in range(N_mc):
        pertubed_params = params.copy()
        for param in uncertain_param_names:
            pertubed_params[param] += np.random.normal(0, stdev)
        pert_res = solve(**pertubed_params)
        res.append(pert_res)

    res = np.array(res)
    # compute percentiles
    percs_res = np.percentile(res, percs, axis=0)
    assert (percs_res.shape == (len(percs), 4, params['N']))
    return percs_res, res


percs_res, res_all = solve_mc(params, stdev)

colors = ['#1b9e77', '#d95f02', '#7570b3']  # colorblind friendly from colorbrewer


# plot 
fig, ax = plt.subplots(1, 1, figsize=[8, 3])
time = np.linspace(0, params['L'], params['N'])
lines_per_percentile = []
for ip in range(len(percs)):
    l1 = ax.plot(time, percs_res[ip, 0], color=colors[0], linestyle=linestyle_percs[ip])
    l2 = ax.plot(time, percs_res[ip, 1], color=colors[1], linestyle=linestyle_percs[ip])
    l3 = ax.plot(time, percs_res[ip, 2], color=colors[2], linestyle=linestyle_percs[ip])
    lines_per_percentile.append([l1, l2, l3])

single_lines = []    
if plot_single_sims:
    for i in range(N_mc):
        l1 = ax.plot(time, res_all[i, 0], color=colors[0], alpha=0.3)
        l2 = ax.plot(time, res_all[i, 1], color=colors[1], alpha=0.3)
        l3 = ax.plot(time, res_all[i, 2], color=colors[2], alpha=0.3)
        single_lines.append([l1,l2,l3])
rect = ax.axvspan(params['t_start_qua'], params['t_stop_qua'], color='0.5', alpha=0.5, linewidth=0, zorder=1)
ax.set_xlabel('time (days)', fontsize=12)
ax.set_ylabel('fraction of population (%)', fontsize=12)
ax.legend(['Susceptible', 'Infected', 'Removed'], loc=(1.1, 0.7))
ax.set_xlim(0, params['L'])
ax.set_xticks(np.arange(0, 400, 30))
ax.set_yticks(np.arange(0, 110, 10))
ax.yaxis.set_ticks_position('both')
ax.tick_params(labeltop=False, labelright=True)
ax.grid(axis='y', linestyle=':', linewidth=0.5)
plt.title(f'{N_mc} simulations')
plt.tight_layout()

fig.canvas.header_visible = False

# Interactive plot
Ro_widget = widgets.FloatSlider(min=0.5, max=6, step=0.25, value=params['Ro'],
                                continuous_update=True, description=r'$R_0$')
tau_widget = widgets.IntSlider(min=1, max=30, step=1, value=params['tau'],
                               continuous_update=True, description=r'$\tau$ (days)')
t_start_qua_widget = widgets.IntSlider(min=0, max=300, step=5, value=params['t_start_qua'],
                                       continuous_update=True, description='$t_\mathrm{start\ qua}$ (day)')
t_stop_qua_widget = widgets.IntSlider(min=0, max=300, step=5, value=params['t_stop_qua'],
                                      continuous_update=True, description='$t_\mathrm{end\ qua}$ (day)')
Ro_qua_widget = widgets.FloatSlider(min=0., max=3., step=0.25, value=params['Ro_qua'],
                                    continuous_update=True, description='$R_{0q}$')
stdev_widget = widgets.FloatSlider(min=0., max=3., step=0.01, value=stdev,
                                    continuous_update=True, description='stdev')

def make_plot(Ro, tau, t_start_qua, t_stop_qua, Ro_qua, stdev):
    # i did not find a way to do this less verbose...
    params['Ro'] = Ro
    params['tau'] = tau
    params['Ro_qua'] = Ro_qua
    params['t_start_qua'] = t_start_qua
    params['t_stop_qua'] = t_stop_qua
    percs_res, res_all = solve_mc(params, stdev)
    for ip in range(len(percs)):
        lines_per_percentile[ip][0][0].set_ydata(percs_res[ip,0])
        lines_per_percentile[ip][1][0].set_ydata(percs_res[ip,1])
        lines_per_percentile[ip][2][0].set_ydata(percs_res[ip,2])
    if plot_single_sims:
        for i in range(N_mc):
            single_lines[i][0][0].set_ydata(res_all[i,0])  
            single_lines[i][1][0].set_ydata(res_all[i,1])  
            single_lines[i][2][0].set_ydata(res_all[i,2])  
    xy = rect.get_xy()
    if t_start_qua < t_stop_qua:
        xy[:, 0] = [t_start_qua, t_start_qua, t_stop_qua, t_stop_qua, t_start_qua]
    else:
        xy[:, 0] = [0, 0, 0.1, 0.1, 0]
    rect.set_xy(xy)


w = interactive(make_plot,
                Ro=Ro_widget,
                tau=tau_widget,
                t_start_qua=t_start_qua_widget,
                t_stop_qua=t_stop_qua_widget,
                Ro_qua=Ro_qua_widget,
                stdev = stdev_widget)

items = w.children
left_box = VBox([items[0], items[1], items[5]])
right_box = VBox([items[2], items[3], items[4]])
HBox([left_box, right_box])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …